# Conectar Python e SQL Server
___

## Definir strings de conexão e criar as 2 funções

In [2]:
import pandas as pd
import numpy as np
import pyodbc
import urllib
from sqlalchemy import event
import sqlalchemy
    
server = '<server_name>.database.windows.net'
database = '<database_name>'
username = '<user_name>'
password = '<password>'
driver= '{ODBC Driver 17 for SQL Server}'
    
def sql_query(query, action='r'):
    # Caso action='r', função vai executar query de leitura e retornar resultado em Pandas DataFrame
    # Caso action ='w' função vai executar query de escrita
    
    if action == 'r':
        with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
            with conn.cursor() as cursor:
                cursor.fast_executemany = True
                output = pd.read_sql_query(query,conn)
                return output
    
    if action == 'w':
        with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
            with conn.cursor() as cursor:
                cursor.fast_executemany = True
                
                cursor.execute(query)
                return print('Query executada')
            
def df_to_sql(df_para_upar, table_name, if_exists="append"):
    # Inserir Pandas DataFrame como tabela em SQL Server Database
        # Parâmetro if_exists:
            # fail: Raise a ValueError.
            # replace: Drop the table before inserting new values.
            # append: Insert new values to the existing table.
    
    db_params = urllib.parse.quote_plus('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))
    @event.listens_for(engine, "before_cursor_execute")
    def receive_before_cursor_execute(
           conn, cursor, statement, params, context, executemany
            ):
                if executemany:
                    cursor.fast_executemany = True
                
    print('Atualizando banco de dados SQL...')
    df_para_upar.to_sql(table_name, engine, index=False, if_exists="append", schema="dbo", chunksize=10000)
    print('Banco de dados atualizado!')   

## Exemplos

In [31]:
# Criar tabela 'PessoaTeste' e inserir 2 registros

query = '''
DROP TABLE IF EXISTS PessoaTeste;

CREATE TABLE PessoaTeste(
Nome VARCHAR(100),
Sobrenome VARCHAR(100)
)

INSERT INTO PessoaTeste VALUES('João', 'Jose');
INSERT INTO PessoaTeste VALUES('Ana', 'Mari');
'''

sql_query(query, 'w')

Query executada


In [33]:
# Consultar tabela 'PessoaTeste'
output = sql_query('select * from PessoaTeste')
output

,Nome,Sobrenome
0,João,Jose
1,Ana,Mari


In [34]:
# Criar 'nova_tabela' a partir de um df
df_to_sql(output, 'nova_tabela')

Atualizando banco de dados SQL...
Banco de dados atualizado!
